In [1]:
%%capture
!pip install catboost

In [1]:
%cd '/content/drive/MyDrive/mchs/emergency_datahack'

from notebooks.notebook_utils import add_project_path
add_project_path()
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import datetime
import features as f 
from match_station import coord_merge, stations_list, coord_km
from optimizers import optimize
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
import datetime
import json

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 200)

SEED = 22

/content/drive/MyDrive/mchs/emergency_datahack


In [2]:
test = pd.read_csv('./data_raw/test.csv', parse_dates=['datetime'])

In [3]:
train = pd.read_pickle('./data_processed/train2020_final_2.pickle.gz')

In [4]:
train = train[~train.datetime.isin(test.datetime)]

In [5]:
X_train, X_val, y_train, y_val = train_test_split(train.drop(['target', 'datetime', 'road_id'], axis=1), 
                                                  train.target,
                                                  shuffle=True,
                                                  stratify=train.target, 
                                                  test_size=0.1, random_state=22)

In [6]:
X_train.head()

,road_km,dist_nearest_traffic_station,lane_count,occupancy_backward,occupancy_forward,speed_backward,speed_forward,volume_backward,volume_forward,lat,lon,vsp_1,vsp_1_q,vsp_2,vsp_2_q,vsp_3,vsp_3_q,visib,visib_q,clouds,clouds_q,weather_range,weather_range_q,weather_on_measure,weather_on_measure_q,wind_dir,wind_dir_q,avg_wind,avg_wind_q,max_wind,max_wind_q,precip,precip_q,temp_on_measure,temp_on_measure_q,temp_min,temp_min_q,temp_max,temp_max_q,humidity,humidity_q,pressure,pressure_q,vsp_mean,vsp_mean_diff,visib_diff,avg_wind_diff,temp_on_measure_diff,precip_diff,humidity_diff,pressure_diff,weather_range_diff,weather_on_measure_diff,weekday,month,hour,s_hour,c_hour,s_month,c_month,cnt_subs,occupancy_backward_diff,occupancy_forward_diff,speed_backward_diff,speed_forward_diff,volume_backward_diff,volume_forward_diff
7438260,800,0.026485,1.0,2.988281,1.855469,66.581253,68.001251,44.0,45.0,65.000000,34.790001,1.0,0.0,1.0,0.0,1.0,0.0,98.0,1.0,12.0,1.0,-1.0,1.0,-1.0,1.0,293.0,1.0,3.000000,1.0,7.0,1.0,0.0,0.0,5.600000,1.0,5.3,1.0,6.4,1.0,64.000000,1.0,1018.599976,1.0,1.076023,0.001949,0.000000,0.000000,0.133333,0.0,-0.333333,0.533333,0.0,0.0,0,4,12,0.000,-1.000,0.866,-0.500,71.0,1.800781,0.445312,-0.748749,1.218750,12.0,-8.0
3697431,507,0.168444,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.169998,34.299999,1.0,0.0,1.0,0.0,1.0,0.0,62.0,0.0,10.0,2.0,-1.0,2.0,-1.0,2.0,221.0,1.0,2.000000,1.0,7.0,1.0,0.0,0.0,-0.066667,1.0,0.3,1.0,1.5,1.0,89.666664,1.0,976.166687,1.0,0.925926,0.037037,0.083333,0.000000,-0.466667,0.0,0.666667,3.166667,0.0,0.0,0,2,22,-0.500,0.866,0.866,0.500,711.0,NaN,NaN,NaN,NaN,NaN,NaN
1133415,587,0.108035,2.0,3.070000,2.685000,63.950001,62.849998,128.0,130.0,59.580002,41.480000,8.0,1.0,18.0,1.0,28.0,1.0,96.0,1.0,10.0,1.0,-1.0,1.0,-1.0,1.0,265.0,1.0,3.000000,1.0,9.0,1.0,0.0,0.0,0.733333,1.0,0.4,1.0,1.0,1.0,94.333336,1.0,985.266663,1.0,18.000000,0.000000,0.333333,0.000000,0.166667,0.0,-0.333333,0.833333,0.0,0.0,4,1,20,-0.866,0.500,0.500,0.866,196.0,-0.455000,-1.550000,-1.149998,-0.400002,-32.0,-78.0
15180639,1335,0.402638,2.0,3.552734,5.343750,59.969376,68.021248,299.0,297.0,68.970001,33.049999,1.0,0.0,1.0,0.0,1.0,0.0,98.0,1.0,12.0,1.0,2.0,1.0,52.0,1.0,39.0,1.0,2.333333,1.0,7.0,1.0,0.0,0.0,10.766666,1.0,9.4,1.0,10.6,1.0,66.666664,1.0,1010.900024,1.0,1.841905,-0.000544,0.000000,0.333333,0.366667,0.0,-1.333333,-0.400000,0.0,0.0,3,8,13,-0.259,-0.966,-0.866,-0.500,171.0,-3.476562,-0.757812,13.815624,9.234997,-7.0,47.0
21057788,1300,0.160753,1.0,0.664062,1.429688,72.526253,75.684998,18.0,46.0,62.919998,34.430000,1.0,1.0,0.0,1.0,0.0,1.0,97.0,1.0,0.0,1.0,9.0,1.0,52.0,1.0,318.0,1.0,11.000000,1.0,17.0,1.0,0.0,0.0,1.800000,1.0,1.8,1.0,2.3,1.0,75.000000,1.0,978.099976,1.0,0.333333,0.000000,0.000000,1.333333,0.500000,0.0,0.000000,-4.333333,1.0,1.0,6,11,21,-0.707,0.707,-0.500,0.866,95.0,-0.671875,-0.656250,-9.848747,-4.357506,-14.0,-30.0


In [7]:
cat_features = [
                # 'road_id', 
                'lane_count', 'vsp_1_q', 'vsp_2_q', 'vsp_3_q', 
                'visib_q', 'clouds', 'clouds_q', 'weather_range', 'weather_range_q',
                'weather_on_measure', 'weather_on_measure_q', 'wind_dir_q',
                'avg_wind_q', 'max_wind_q', 'precip_q', 'temp_on_measure_q', 
                'temp_min_q', 'temp_max_q', 'humidity_q', 'pressure_q', 
                'weather_range_diff', 'weather_on_measure_diff', 'weekday', 'month', 'hour',
                ]

In [8]:
for f in cat_features:
  X_train[f].fillna(-1, inplace=True)
  X_train[f] = X_train[f].astype(int)

  X_val[f].fillna(-1, inplace=True)
  X_val[f] = X_val[f].astype(int)

In [ ]:
train_pool = Pool(X_train, 
                  label=y_train,
                  cat_features=cat_features,)

In [ ]:
val_pool = Pool(X_val, 
                  label=y_val,
                  cat_features=cat_features)

In [ ]:
del train, test, X_train, X_val, y_train, y_val

In [ ]:
pd.Series(train_pool.get_label()).value_counts()

In [ ]:
q_features = [f for f in cat_features if '_q' in f]

In [ ]:
params = {
    'ignored_features': q_features,
    'iterations': 2000,
    'learning_rate': 0.2,
    'class_weights': [0.01, 1, 2],
    # 'min_data_in_leaf': 1,
    'one_hot_max_size': 255,
    'depth': 8,
    'loss_function': 'MultiClass',
    'task_type': 'GPU',
    'use_best_model': True,
    'eval_metric': 'TotalF1:weighted=false;average=Macro',
    'random_seed': SEED,
    'has_time': True
}


In [ ]:
cls = CatBoostClassifier(**params)

In [ ]:
cls.fit(
    train_pool,
    eval_set=val_pool,
    plot=True,
    verbose=10,
    early_stopping_rounds=20,
)

In [44]:
pd.DataFrame({'feature_importance': cls.feature_importances_, 
              'feature_names': cls.feature_names_,}).sort_values(by=['feature_importance'], ascending=False).head(20)

,feature_importance,feature_names
67,16.849951,historical_counter
60,4.479270,cnt_subs
56,3.858669,s_hour
39,3.592198,humidity
1,3.446758,dist_nearest_traffic_station
53,3.235313,weekday
41,3.198246,pressure
57,3.154251,c_hour
55,2.995276,hour
54,2.909362,month


In [45]:
pd.Series(cls.predict(val_pool).ravel()).value_counts()

0    2284777
1       3563
2          5
dtype: int64

In [46]:
now = datetime.datetime.now()
cls.save_model(f'./results/cb_model_{now}.pt')
with open(f'./results/params_{now}.json', 'w') as fp:
    json.dump(params, fp)